In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings

# 1. Data Preparation

In [3]:
trainDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_train_cbMSIn0d14nonInDI_norm.csv")
diDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_nonInDI_cbMSIn0d14nonInDI_norm.csv")

In [4]:
trainDEFSDf

,pixel_id,311.15,269.25,215.0,295.1,883.55,309.2,738.55,435.2,280.25,241.1,311.2,339.1,353.2,325.2,250.1,514.3,265.05,type
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.306614,0.155823,-0.014750,0.167718,-0.083734,0.256635,-0.155483,0.117061,-0.083208,0.218163,0.306614,0.224674,0.269023,0.292024,0.125619,-0.041124,0.319844,0
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.426535,0.413081,0.099901,0.288114,-0.052378,0.340275,-0.127744,0.057160,-0.103060,0.283815,0.426535,0.306224,0.329107,0.397299,0.299930,-0.041124,0.387575,0
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.283540,0.172545,0.049710,0.137600,-0.081307,0.200232,-0.162255,-0.030521,-0.088757,0.202159,0.283540,0.223596,0.244158,0.266221,0.208253,0.104304,0.310121,0
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.264706,0.118793,-0.002867,0.190098,-0.066417,0.197218,-0.155254,-0.019154,-0.107369,0.264374,0.264706,0.214958,0.196900,0.245780,0.199638,-0.041124,0.298688,0
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.162278,0.208302,-0.049377,0.145323,-0.064760,0.118803,-0.158031,0.138977,-0.065062,0.158361,0.162278,0.133026,0.183183,0.150113,0.094244,-0.041124,0.214619,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.021175,-0.289340,-0.057136,-0.066866,-0.037735,0.020800,-0.207775,0.241656,-0.174294,-0.257142,0.021175,0.037142,0.060526,0.034232,0.037630,0.212195,0.060493,1
90956,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.007508,-0.289340,-0.152535,-0.066866,-0.074200,0.032496,-0.090080,0.299395,-0.195733,0.144289,0.007508,0.044634,0.064259,0.036020,0.210084,0.134625,0.033504,1
90957,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.016589,0.347700,-0.094148,-0.066866,-0.102274,0.050359,-0.070099,0.194703,-0.114568,-0.257142,0.016589,0.041114,0.095722,0.036409,0.142757,-0.041124,-0.057701,1
90958,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.031206,-0.289340,-0.090194,-0.066866,-0.081719,0.027120,-0.055867,0.148581,-0.163662,-0.257142,0.031206,0.028236,0.055537,0.035391,0.188492,-0.041124,0.060124,1


In [5]:
can = []
for i in list(range(0,9)) + list(range(15,17)) + [len(list(trainDEFSDf.columns))-1]:
    can.append(list(trainDEFSDf.columns)[i])

dican = []
for i in list(range(0,9)) + list(range(15,17)) + [len(list(diDEFSDf.columns))-1]:
    dican.append(list(diDEFSDf.columns)[i])

In [6]:
can

['pixel_id',
 '311.15',
 '269.25',
 '215.0',
 '295.1',
 '883.55',
 '309.2',
 '738.55',
 '435.2',
 '250.1',
 '514.3',
 'type']

In [7]:
dican

['pixel_id',
 '311.2',
 '269.2',
 '215.0',
 '295.2',
 '883.6',
 '309.2',
 '738.6',
 '435.2',
 '250.1',
 '514.2',
 'type']

In [8]:
## input training set ## 90960 x 20 df
trainDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_train_cbMSIn0d14nonInDI_norm.csv")
#trainDEFSDf[trainDEFSDf.type .== 1, :]
## calculate weight ## 0: 47449, 1: 43511
trainDEFSDf = trainDEFSDf[can]
Yy_train = trainDEFSDf["type"]  # 0.9585; 1.0453
sampleW = []
for w in Yy_train:
    if w == 0:
        sampleW.append(0.9585)
    elif w == 1:
        sampleW.append(1.0453) 


In [9]:
## input ext val set ## 6075 x 20 df
extDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ext_cbMSIn0d14nonInDI_norm.csv")
#extDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 2943, 1: 3132
extDEFSDf = extDEFSDf[can]
Yy_ext = extDEFSDf["type"]  # 1.0321; 0.9698
sampleExtW = []
for w in Yy_ext:
    if w == 0:
        sampleExtW.append(1.0321)
    elif w == 1:
        sampleExtW.append(0.9698) 


In [10]:
## input ingested set ## 97035 x 20 df
ingestedDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ingested_cbMSIn0d14nonInDI_norm.csv")
#ingestedDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 50392, 1: 46643
ingestedDEFSDf = ingestedDEFSDf[can]
Yy_ingested = ingestedDEFSDf["type"]  # 0.9628; 1.0402
sampleIngestedW = []
for w in Yy_ingested:
    if w == 0:
        sampleIngestedW.append(0.9628)
    elif w == 1:
        sampleIngestedW.append(1.0402)


In [11]:
## input FNA set ## 88701 x 20 df
fnaDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_FNA_cbMSIn0d14nonInDI_norm.csv")
#fnaDEFSDf[fnaDEFSDf.type .== 1, :]
## calculate weight ##  0: 44540, 1: 44161
fnaDEFSDf = fnaDEFSDf[can]
Yy_FNA = fnaDEFSDf["type"]  # 0.9957; 1.0043
sampleFNAW = []
for w in Yy_FNA:
    if w == 0:
        sampleFNAW.append(0.9957)
    elif w == 1:
        sampleFNAW.append(1.0043)


In [12]:
## input DirectIn set ## 88701 x 20 df
diDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_nonInDI_cbMSIn0d14nonInDI_norm.csv")
#diDEFSDf[diDEFSDf.type .== 1, :]
## calculate weight ## 0: 3027, 1: 3030
diDEFSDf = diDEFSDf[dican]
diDEFSDf = diDEFSDf.rename(columns={"311.2": "311.15", "269.2":"269.25", "295.2":"295.1", "883.6":"883.55", 
                                    "738.6":"738.55", "280.2":"280.25", 
                                    "339.2":"339.1", 
                                    "514.2":"514.3", "265.2":"265.05"})
Yy_DI = diDEFSDf["type"]  # 1.0005; 0.9995
sampleDiW = []
for w in Yy_DI:
    if w == 0:
        sampleDiW.append(1.0005)
    elif w == 1:
        sampleDiW.append(0.9995)


In [13]:
diDEFSDf

,pixel_id,311.15,269.25,215.0,295.1,883.55,309.2,738.55,435.2,250.1,514.3,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,-0.000522,-0.016303,0.027673,0.021522,0.045789,0.116087,0.061222,0.010616,0.437911,0.036473,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.083005,0.027506,0.116839,0.031732,0.127053,0.216725,0.134751,0.003907,0.553596,0.099593,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.025963,-0.010108,0.011686,-0.046683,0.118207,0.127443,0.028351,0.052377,0.434098,0.056801,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.074431,0.087626,0.059979,0.071668,0.092430,0.203181,0.107481,0.083333,0.520221,0.090314,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.128189,0.110326,0.113772,0.132852,0.193001,0.280986,0.235611,0.124496,0.605425,0.147309,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.005316,-0.009320,0.122463,-0.019665,0.019314,-0.015528,0.022632,0.150664,-0.037582,0.163243,1
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.313499,-0.280980,-0.221368,-0.290503,-0.274617,-0.372822,-0.281165,-0.334752,-0.035654,-0.250800,1
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.142502,-0.014532,0.041403,-0.055275,-0.066586,-0.108891,-0.085526,-0.066395,-0.129441,0.091504,1
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.126710,-0.072820,0.097524,-0.092329,-0.003292,-0.106890,-0.069614,-0.061199,0.005436,-0.018064,1


In [14]:
## define functions for performace evaluation ##

# Average score
def avgScore(arrAcc, cv):
    sumAcc = 0
    for acc in arrAcc:
        sumAcc += acc
    return sumAcc / cv


In [15]:
trainDEFSDf[trainDEFSDf.columns[1:-1]]

,311.15,269.25,215.0,295.1,883.55,309.2,738.55,435.2,250.1,514.3
0,0.306614,0.155823,-0.014750,0.167718,-0.083734,0.256635,-0.155483,0.117061,0.125619,-0.041124
1,0.426535,0.413081,0.099901,0.288114,-0.052378,0.340275,-0.127744,0.057160,0.299930,-0.041124
2,0.283540,0.172545,0.049710,0.137600,-0.081307,0.200232,-0.162255,-0.030521,0.208253,0.104304
3,0.264706,0.118793,-0.002867,0.190098,-0.066417,0.197218,-0.155254,-0.019154,0.199638,-0.041124
4,0.162278,0.208302,-0.049377,0.145323,-0.064760,0.118803,-0.158031,0.138977,0.094244,-0.041124
...,...,...,...,...,...,...,...,...,...,...
90955,0.021175,-0.289340,-0.057136,-0.066866,-0.037735,0.020800,-0.207775,0.241656,0.037630,0.212195
90956,0.007508,-0.289340,-0.152535,-0.066866,-0.074200,0.032496,-0.090080,0.299395,0.210084,0.134625
90957,0.016589,0.347700,-0.094148,-0.066866,-0.102274,0.050359,-0.070099,0.194703,0.142757,-0.041124
90958,0.031206,-0.289340,-0.090194,-0.066866,-0.081719,0.027120,-0.055867,0.148581,0.188492,-0.041124


In [16]:
import warnings
warnings.filterwarnings('ignore')

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [17]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [18]:
df_ROI_for_ML_Opti = pd.concat([trainDEFSDf, extDEFSDf, fnaDEFSDf, diDEFSDf]).set_index('pixel_id')

In [19]:
df_ROI_for_ML_Opti_ingested = ingestedDEFSDf.set_index('pixel_id')

In [20]:
df_ROI_for_ML_Opti_train = trainDEFSDf.set_index('pixel_id')

In [21]:
df_ROI_for_ML_Opti_ext = extDEFSDf.set_index('pixel_id')

In [22]:
df_ROI_for_ML_Opti_FNA = fnaDEFSDf.set_index('pixel_id')

In [23]:
df_ROI_for_ML_Opti_di = diDEFSDf.set_index('pixel_id')

In [24]:
df_ROI_for_ML_Opti_di.describe()

,311.15,269.25,215.0,295.1,883.55,309.2,738.55,435.2,250.1,514.3,type
count,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6057.000000
mean,-4.303787e-17,6.686633e-17,-1.367845e-16,-1.968598e-17,-8.858689e-17,1.546376e-16,7.654435e-17,-6.484091e-17,5.154738e-16,-5.315580e-19,0.500248
std,1.244602e-01,1.225812e-01,1.425257e-01,1.355446e-01,1.755429e-01,1.525340e-01,1.278292e-01,1.785266e-01,1.882975e-01,1.329823e-01,0.500041
min,-4.080066e-01,-4.162738e-01,-3.567420e-01,-4.723469e-01,-2.746170e-01,-4.499012e-01,-2.811650e-01,-3.347515e-01,-1.294412e-01,-3.328045e-01,0.000000
25%,-7.203839e-02,-4.670358e-02,-7.034140e-02,-5.548548e-02,-9.740304e-02,-6.680087e-02,-6.020093e-02,-1.225706e-01,-1.294412e-01,-6.636153e-02,0.000000
50%,1.038792e-02,1.566504e-02,9.645582e-03,1.623459e-02,2.527424e-02,-7.133968e-03,1.495291e-02,-7.940668e-03,-3.531219e-02,-1.855730e-02,1.000000
75%,8.697130e-02,7.564939e-02,8.470268e-02,7.445151e-02,1.230419e-01,5.221173e-02,6.986257e-02,1.154688e-01,1.845510e-02,8.936343e-02,1.000000
max,5.919934e-01,5.837262e-01,6.432580e-01,5.276531e-01,7.253830e-01,5.500988e-01,7.188350e-01,6.652485e-01,8.705588e-01,6.671955e-01,1.000000


## 2.2. PCA Analysis

In [37]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_di):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_di.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1', size=20)
    plt.ylabel('PC2', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/cbMSIn_0d1/PCA_di.tif", bbox_inches = 'tight')

    components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()

In [ ]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [39]:
def split_dataset(df=df_ROI_for_ML_Opti_ingested):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2,random_state=42)
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti)

In [40]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]]

In [41]:
y_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[-1]]

In [42]:
X_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[:-1]]

In [43]:
y_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[-1]]

In [44]:
X_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[:-1]]

In [45]:
y_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[-1]]

In [46]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [47]:
y_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[-1]]

In [48]:
X_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[:-1]]

In [49]:
y_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[-1]]

## 3.2 Machine Learning Modeling: Model 1+2 (Train 10:10, Val 4:4, Test 3:3)

In [50]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### 3.2.1. Logistic Regression

In [51]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [52]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.8408210677463104
Training Set MCC: 0.6836806332505082
Training Set Accuracy: 0.8420844327176781
              precision    recall  f1-score   support

           0       0.85      0.85      0.85     47449
           1       0.83      0.84      0.84     43511

    accuracy                           0.84     90960
   macro avg       0.84      0.84      0.84     90960
weighted avg       0.84      0.84      0.84     90960

[[40240  7209]
 [ 7155 36356]]
Ext Val Set F1: 0.7663903187532383
Ext Val Set MCC: 0.4917921638077761
Ext Val Set Accuracy: 0.7427160493827161
              precision    recall  f1-score   support

           0       0.80      0.62      0.70      2943
           1       0.71      0.86      0.77      3132

    accuracy                           0.74      6075
   macro avg       0.75      0.74      0.74      6075
weighted avg       0.75      0.74      0.74      6075

[[1831 1112]
 [ 451 2681]]
5F-CV: 0.5934811363879445
FNA Set F1: 0.8023948911234491
FNA 

### 3.2.2. SVM

In [53]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [54]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.8625276945774187
Training Set MCC: 0.7222981688430185
Training Set Accuracy: 0.8606090589270009
              precision    recall  f1-score   support

           0       0.88      0.85      0.86     47449
           1       0.84      0.87      0.86     43511

    accuracy                           0.86     90960
   macro avg       0.86      0.86      0.86     90960
weighted avg       0.86      0.86      0.86     90960

[[40354  7095]
 [ 5584 37927]]
Ext Val Set F1: 0.7002656056685465
Ext Val Set MCC: 0.3343660885401798
Ext Val Set Accuracy: 0.665514403292181
              precision    recall  f1-score   support

           0       0.71      0.53      0.61      2943
           1       0.64      0.79      0.71      3132

    accuracy                           0.67      6075
   macro avg       0.67      0.66      0.66      6075
weighted avg       0.67      0.67      0.66      6075

[[1566 1377]
 [ 655 2477]]
5F-CV: 0.818769918713504
FNA Set F1: 0.7307993360907897
FNA Se

### 3.2.3. GBM

In [55]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [56]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.6472767285845505
Training Set MCC: 0.27144438301690627
Training Set Accuracy: 0.6339599824098505
              precision    recall  f1-score   support

           0       0.66      0.60      0.63     47449
           1       0.61      0.67      0.64     43511

    accuracy                           0.63     90960
   macro avg       0.64      0.64      0.63     90960
weighted avg       0.64      0.63      0.63     90960

[[28555 18894]
 [14401 29110]]
Ext Val Set F1: 0.2282017391299275
Ext Val Set MCC: -0.3829667931547765
Ext Val Set Accuracy: 0.30979423868312755
              precision    recall  f1-score   support

           0       0.33      0.42      0.37      2943
           1       0.27      0.20      0.23      3132

    accuracy                           0.31      6075
   macro avg       0.30      0.31      0.30      6075
weighted avg       0.30      0.31      0.30      6075

[[1246 1697]
 [2496  636]]
5F-CV: 0.6749133305388579
FNA Set F1: 0.3950306722919045
F

### 3.2.4. Stacking

In [57]:
class StackingRegressor():
    
    def __init__(self, learners):
        self.level_sizes = []
        self.learners = []
        
        for learning_level in learners:
            self.level_sizes.append(len(learning_level))
            level_learners = []
            
            for learner in learning_level:
                level_learners.append(deepcopy(learner))
            self.learners.append(level_learners)
            
            
    def fit(self, x, y):
        
        meta_data = [x,x,x]
        meta_targets = [y,y,y]
        
        for i in range(len(self.learners)):
            level_size = self.level_sizes[i]
            
            data_z = np.zeros((level_size, len(x)))
            target_z = np.zeros(len(x))
            
            train_x = meta_data[i]
            train_y = meta_targets[i]
            
            # Define number of folds
            num_folds = 5
            
            # Create the k-fold cross-validation object
            KF = KFold(n_splits=num_folds)
            m = 0

            # Loop over each fold of the cross-validation
            for train_indices, test_indices in KF.split(x):
                
                for j in range(len(self.learners[i])):
                    train_x = pd.DataFrame(train_x)
                    train_y = pd.DataFrame(train_y)

                    learner = self.learners[i][j]
                    learner.fit(train_x.iloc[train_indices], train_y.iloc[train_indices])
                    p = learner.predict(train_x.iloc[test_indices])
                    data_z[j][m: m+len(test_indices) ] = p


                train_y = pd.DataFrame(train_y)
                train_y_array = (np.array(train_y)).reshape(-1,)
                zty_ind = (np.array(train_y.iloc[test_indices])).reshape(-1,)
                target_z[m: m+len(test_indices)] = train_y_array[zty_ind]
                m += len(test_indices)
                

            data_z = data_z.transpose()
            meta_data.append(data_z)
            meta_targets.append(target_z)
            
            
            for learner in self.learners[i]:
                train_x = pd.DataFrame(train_x)
                train_y = pd.DataFrame(train_y)
                learner.fit(train_x, train_y)
                
                
    def predict(self, x):
        
        meta_data = [x,x,x]
        for i in range(len(self.learners)):
            
            level_size = self.level_sizes[i]
            
            data_z = np.zeros((level_size, len(x)))
            
            test_x = meta_data[i]
            
            for j in range(len(self.learners[i])):
                
                learner = self.learners[i][j]
                predictions = learner.predict(test_x)
                data_z[j] = predictions
                
            
            data_z = data_z.transpose()
            meta_data.append(data_z)
            
        return meta_data


In [58]:
## Build stacking Model

def Stacking_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    #meta_learner = LinearDiscriminantAnalysis()  # 0.90,0.80; 0.75,0.51; 0.65,0.27; 0.64,0.26
    #meta_learner = QuadraticDiscriminantAnalysis()  # 0.93,0.86; 0.67,0.39; 0.67,0.15; 0.68,0.20
    
    #meta_learner = LogisticRegression()  # 0.90,0.79; 0.82,0.65; 0.64,0.24; 0.64;0.27
    #meta_learner = LogisticRegression(C=1, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=1: 0.90,0.79; 0.82,0.65; 0.65,0.25; 0.65;0.27
    #meta_learner = LogisticRegression(C=0.1, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.1: 0.90,0.79; 0.82,0.65; 0.65,0.25; 0.65;0.28
    #meta_learner = LogisticRegression(C=0.01, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.01: 0.89,0.78; 0.81,0.63; 0.63,0.22; 0.65;0.29
    #meta_learner = LogisticRegression(C=0.0075, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0075: 0.89,0.77; 0.81,0.63; 0.62,0.20; 0.65;0.29
    #meta_learner = LogisticRegression(C=0.005, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.005: 0.88,0.76; 0.82,0.64; 0.60,0.17; 0.66;0.31
    #meta_learner = LogisticRegression(C=0.0025, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0025: 0.86,0.72; 0.83,0.66; 0.53,0.02; 0.64;0.31
    #meta_learner = LogisticRegression(C=0.0015, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0015: 0.85,0.70; 0.80,0.60; 0.55,0.09; 0.64;0.30
    #meta_learner = LogisticRegression(C=0.0013, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0013: 0.85,0.70; 0.78,0.56; 0.58,0.16; 0.63;0.29
    #meta_learner = LogisticRegression(C=0.0012, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0012: 0.85,0.69; 0.77,0.52; 0.60,0.22; 0.63;0.27
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0011: 0.85,0.69; 0.77,0.51; 0.64,0.30; 0.62;0.25
    #meta_learner = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.001: 0.84,0.68; 0.76,0.49; 0.68,0.38; 0.61;0.22
    #meta_learner = LogisticRegression(C=0.0005, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0005: 0.83,0.66; 0.74,0.45; 0.65,0.37; 0.60;0.18
    
    #meta_learner = LinearSVC()  # 0.90,0.80; 0.82,0.65; 0.64,0.24; 0.65,0.27
    #meta_learner = KNeighborsClassifier()  # 0.98,0.96; 0.65,0.30; 0.68,0.21; 0.52,-0.05
    #meta_learner = DecisionTreeClassifier()  # 0.94,0.87; 0.54,0.15; 0.66,0.11; 0.46,-0.11
    #meta_learner = RandomForestClassifier()  # 0.98,0.95; 0.54,0.20; 0.66,0.11; 0.49;-0.01
    #meta_learner = GradientBoostingClassifier()  # 0.97,0.94; 0.56,0.27; 0.66,0.13; 0.48,-0.09
    #meta_learner = AdaBoostClassifier()  # 0.96,0.93; 0.69,0.42; 0.62,0.10; 0.54,0.11
    #meta_learner = xgb.XGBClassifier()  # 0.98,0.95; 0.59,0.30; 0.66,0.10; 0.44,-0.12

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(X_train)[-1]
    print("Training Set F1:", f1_score(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XVal)[-1]
    print("Ext Val Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [59]:
Stacking_func()

Training Set F1: 0.841512992702823
Training Set MCC: 0.6850278433560042
Training Set Accuracy: 0.8427550571679859
              precision    recall  f1-score   support

           0       0.85      0.85      0.85     47449
           1       0.84      0.84      0.84     43511

    accuracy                           0.84     90960
   macro avg       0.84      0.84      0.84     90960
weighted avg       0.84      0.84      0.84     90960

[[40268  7181]
 [ 7122 36389]]
Ext Val Set F1: 0.7699649280543268
Ext Val Set MCC: 0.49940411529781636
Ext Val Set Accuracy: 0.7461728395061729
              precision    recall  f1-score   support

           0       0.81      0.62      0.70      2943
           1       0.71      0.86      0.78      3132

    accuracy                           0.75      6075
   macro avg       0.76      0.74      0.74      6075
weighted avg       0.76      0.75      0.74      6075

[[1833 1110]
 [ 432 2700]]
FNA Set F1: 0.7999649183922877
FNA Set MCC: 0.603633262987522

### 3.2.5. Voting

In [60]:
# get a voting ensemble of models
def get_voting():
    models = list()
    # Define the base models
    models.append(("m1", LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    models.append(("m2", LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    models.append(("m3", GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)))

    # define the voting ensemble
    ensemble = VotingClassifier(estimators=models, voting='hard')
    return ensemble

# get a list of models to evaluate
def get_models():
    models = dict()
    models['m1'] = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    models['m2'] = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    models['m3'] = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    models['hard_voting'] = get_voting()
    return models


# evaluate a give model using cross-validation
def evaluate_model(model, XX, yy):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
    scores = cross_val_score(model, XX, yy, scoring='f1', cv=cv, n_jobs=-1, error_score='raise')
    return scores


In [61]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [63]:
# get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, XX=X_ingested, yy=y_ingested)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
    
# plot model performance for comparison
plt.subplots(dpi = 600)
plt.boxplot(results, labels=names, showmeans=True)
plt.ylabel('F1 score')
plt.xlabel('Predictive Models')
plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/cbMSIn_0d1/voting.jpg", bbox_inches = 'tight')

>m1 0.771 (0.003)
>m2 0.886 (0.020)
>m3 0.664 (0.143)
>hard_voting 0.860 (0.034)


In [64]:
def Voting_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(X_train)
    print("Training Set F1:", f1_score(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [65]:
Voting_func()

Training Set F1: 0.8457539849027802
Training Set MCC: 0.6927689857504232
Training Set Accuracy: 0.8465479331574318
              precision    recall  f1-score   support

           0       0.85      0.85      0.85     47449
           1       0.84      0.84      0.84     43511

    accuracy                           0.85     90960
   macro avg       0.85      0.85      0.85     90960
weighted avg       0.85      0.85      0.85     90960

[[40351  7098]
 [ 6860 36651]]
Ext Val Set F1: 0.6803762000283127
Ext Val Set MCC: 0.3044493313069797
Ext Val Set Accuracy: 0.6523456790123456
              precision    recall  f1-score   support

           0       0.67      0.55      0.61      2943
           1       0.64      0.75      0.69      3132

    accuracy                           0.65      6075
   macro avg       0.66      0.65      0.65      6075
weighted avg       0.66      0.65      0.65      6075

[[1625 1318]
 [ 794 2338]]
5F-CV: 0.7848617768682282
FNA Set F1: 0.7220408506086006
FNA 

In [66]:
import joblib as jl

In [67]:
models = get_models()
    
ensemble = models["hard_voting"]
ensemble.fit(X_ingested, y_ingested)

VotingClassifier(estimators=[('m1',
                              LogisticRegression(C=0.001,
                                                 class_weight={0: 0.9628,
                                                               1: 1.0402},
                                                 max_iter=5000, penalty='l1',
                                                 random_state=42,
                                                 solver='liblinear')),
                             ('m2',
                              LinearSVC(C=461,
                                        class_weight={0: 0.9628, 1: 1.0402},
                                        random_state=42)),
                             ('m3',
                              GradientBoostingClassifier(learning_rate=4,
                                                         max_depth=7,
                                                         min_samples_leaf=4,
                                                         min_sampl

In [68]:
modelSavePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d1\model3to1Voting_cbMSIn0d1.joblib"
jl.dump(ensemble, modelSavePath, compress = 5)

['H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\cbMSIn_0d1\\model3to1Voting_cbMSIn0d1.joblib']

## 3.3 Machine Learning Modeling: Model 1 (Train 6:6, Val 4:4, Test 3:3)

### 3.3.1. Logistic Regression

In [69]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [70]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.8569485297715077
Training Set MCC: 0.7121772157313848
Training Set Accuracy: 0.8557937554969217
              precision    recall  f1-score   support

           0       0.87      0.85      0.86     47449
           1       0.84      0.86      0.85     43511

    accuracy                           0.86     90960
   macro avg       0.86      0.86      0.86     90960
weighted avg       0.86      0.86      0.86     90960

[[40326  7123]
 [ 5994 37517]]
Ext Val Set F1: 0.6150514329102017
Ext Val Set MCC: 0.17088791678007098
Ext Val Set Accuracy: 0.5868312757201646
              precision    recall  f1-score   support

           0       0.59      0.50      0.54      2943
           1       0.59      0.66      0.62      3132

    accuracy                           0.59      6075
   macro avg       0.59      0.58      0.58      6075
weighted avg       0.59      0.59      0.58      6075

[[1485 1458]
 [1052 2080]]
5F-CV: 0.7364561480445152
FNA Set F1: 0.7497707260607065
FNA

### 3.3.2. SVM

In [71]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [72]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.9109933543289993
Training Set MCC: 0.8288662702072551
Training Set Accuracy: 0.9149846086191733
              precision    recall  f1-score   support

           0       0.90      0.94      0.92     47449
           1       0.94      0.88      0.91     43511

    accuracy                           0.91     90960
   macro avg       0.92      0.91      0.91     90960
weighted avg       0.92      0.91      0.91     90960

[[44785  2664]
 [ 5069 38442]]
Ext Val Set F1: 0.2380177560743006
Ext Val Set MCC: -0.44191610821732286
Ext Val Set Accuracy: 0.27868312757201646
              precision    recall  f1-score   support

           0       0.29      0.34      0.31      2943
           1       0.26      0.22      0.24      3132

    accuracy                           0.28      6075
   macro avg       0.28      0.28      0.28      6075
weighted avg       0.28      0.28      0.28      6075

[[ 989 1954]
 [2428  704]]
5F-CV: 0.8550142758110824
FNA Set F1: 0.5486344047360954
F

### 3.3.3. GBM

In [73]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [74]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.8113686759633589
Training Set MCC: 0.6189844161805929
Training Set Accuracy: 0.8089819700967458
              precision    recall  f1-score   support

           0       0.83      0.80      0.81     47449
           1       0.79      0.82      0.80     43511

    accuracy                           0.81     90960
   macro avg       0.81      0.81      0.81     90960
weighted avg       0.81      0.81      0.81     90960

[[37919  9530]
 [ 7845 35666]]
Ext Val Set F1: 0.17594419243837597
Ext Val Set MCC: -0.35853111482604305
Ext Val Set Accuracy: 0.328559670781893
              precision    recall  f1-score   support

           0       0.37      0.53      0.43      2943
           1       0.24      0.14      0.18      3132

    accuracy                           0.33      6075
   macro avg       0.30      0.33      0.31      6075
weighted avg       0.30      0.33      0.30      6075

[[1551 1392]
 [2687  445]]
5F-CV: 0.613404446652406
FNA Set F1: 0.17822201029834633
FN

### 3.3.4. Stacking

In [75]:
## Build stacking Model

def Stacking_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(X_train)[-1]
    print("Training Set F1:", f1_score(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XVal)[-1]
    print("Ext Val Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [76]:
Stacking_func()

Training Set F1: 0.8581549740450832
Training Set MCC: 0.7143628683390962
Training Set Accuracy: 0.8568381706244503
              precision    recall  f1-score   support

           0       0.87      0.85      0.86     47449
           1       0.84      0.86      0.85     43511

    accuracy                           0.86     90960
   macro avg       0.86      0.86      0.86     90960
weighted avg       0.86      0.86      0.86     90960

[[40334  7115]
 [ 5907 37604]]
Ext Val Set F1: 0.5914788118085019
Ext Val Set MCC: 0.12710476188572267
Ext Val Set Accuracy: 0.5651028806584362
              precision    recall  f1-score   support

           0       0.56      0.49      0.52      2943
           1       0.57      0.63      0.60      3132

    accuracy                           0.57      6075
   macro avg       0.56      0.56      0.56      6075
weighted avg       0.56      0.57      0.56      6075

[[1451 1492]
 [1150 1982]]
FNA Set F1: 0.7454350055251271
FNA Set MCC: 0.48789172860553

### 3.3.5. Voting

In [77]:
def Voting_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(X_train)
    print("Training Set F1:", f1_score(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [78]:
Voting_func()

Training Set F1: 0.9218312799074067
Training Set MCC: 0.8471903324479612
Training Set Accuracy: 0.9241094986807388
              precision    recall  f1-score   support

           0       0.91      0.94      0.93     47449
           1       0.93      0.90      0.92     43511

    accuracy                           0.92     90960
   macro avg       0.92      0.92      0.92     90960
weighted avg       0.92      0.92      0.92     90960

[[44700  2749]
 [ 4154 39357]]
Ext Val Set F1: 0.26912856862750617
Ext Val Set MCC: -0.3291250566775732
Ext Val Set Accuracy: 0.3354732510288066
              precision    recall  f1-score   support

           0       0.35      0.43      0.39      2943
           1       0.31      0.24      0.27      3132

    accuracy                           0.34      6075
   macro avg       0.33      0.34      0.33      6075
weighted avg       0.33      0.34      0.33      6075

[[1275 1668]
 [2369  763]]
5F-CV: 0.8250353689979267
FNA Set F1: 0.5642922155848031
FN

## 3.4 Machine Learning Modeling: Model 2 (Train 4:4, Val 6:6, Test 3:3)

### 3.4.1. Logistic Regression

In [79]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_lr.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_lr.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [80]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.0
Training Set MCC: 0.0
Training Set Accuracy: 0.5216468777484609
              precision    recall  f1-score   support

           0       0.52      1.00      0.69     47449
           1       0.00      0.00      0.00     43511

    accuracy                           0.52     90960
   macro avg       0.26      0.50      0.34     90960
weighted avg       0.27      0.52      0.36     90960

[[47449     0]
 [43511     0]]
Ext Val Set F1: 0.0
Ext Val Set MCC: 0.0
Ext Val Set Accuracy: 0.48444444444444446
              precision    recall  f1-score   support

           0       0.48      1.00      0.65      2943
           1       0.00      0.00      0.00      3132

    accuracy                           0.48      6075
   macro avg       0.24      0.50      0.33      6075
weighted avg       0.23      0.48      0.32      6075

[[2943    0]
 [3132    0]]
5F-CV: 0.0
FNA Set F1: 0.0
FNA Set MCC: 0.0
FNA Set Recall: 0.0
FNA Test Set Accuracy: 0.5021363907960451
              

### 3.4.2. SVM

In [81]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [82]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.822060296453499
Training Set MCC: 0.6471318805689714
Training Set Accuracy: 0.8238786279683378
              precision    recall  f1-score   support

           0       0.83      0.83      0.83     47449
           1       0.82      0.82      0.82     43511

    accuracy                           0.82     90960
   macro avg       0.82      0.82      0.82     90960
weighted avg       0.82      0.82      0.82     90960

[[39466  7983]
 [ 8037 35474]]
Ext Val Set F1: 0.9496782743961818
Ext Val Set MCC: 0.9012093799913425
Ext Val Set Accuracy: 0.9499588477366255
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      2943
           1       0.97      0.94      0.95      3132

    accuracy                           0.95      6075
   macro avg       0.95      0.95      0.95      6075
weighted avg       0.95      0.95      0.95      6075

[[2840  103]
 [ 201 2931]]
5F-CV: 0.8823789778869807
FNA Set F1: 0.6746413925803055
FNA S

### 3.4.3. GBM

In [83]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_gbm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_gbm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [84]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.5414647738806154
Training Set MCC: 0.2041944147732082
Training Set Accuracy: 0.6042656112576957
              precision    recall  f1-score   support

           0       0.60      0.72      0.66     47449
           1       0.61      0.47      0.53     43511

    accuracy                           0.60     90960
   macro avg       0.61      0.60      0.60     90960
weighted avg       0.61      0.60      0.60     90960

[[34353 13096]
 [22900 20611]]
Ext Val Set F1: 0.6055917548901529
Ext Val Set MCC: 0.2326140950798751
Ext Val Set Accuracy: 0.6153086419753087
              precision    recall  f1-score   support

           0       0.60      0.64      0.62      2943
           1       0.64      0.59      0.61      3132

    accuracy                           0.62      6075
   macro avg       0.62      0.62      0.62      6075
weighted avg       0.62      0.62      0.62      6075

[[1892 1051]
 [1286 1846]]
5F-CV: 0.6467367938622762
FNA Set F1: 0.559634571140626
FNA S

### 3.4.4. Stacking

In [85]:
## Build stacking Model

def Stacking_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(XVal)[-1]
    print("Training Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XTrain)[-1]
    print("Ext Val Set F1:", f1_score(yTrain, model_sc.predict(XTrain)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_sc.predict(XTrain)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [86]:
Stacking_func()

Training Set F1: 0.0
Training Set MCC: 0.0
Training Set Accuracy: 0.5216468777484609
              precision    recall  f1-score   support

           0       0.52      1.00      0.69     47449
           1       0.00      0.00      0.00     43511

    accuracy                           0.52     90960
   macro avg       0.26      0.50      0.34     90960
weighted avg       0.27      0.52      0.36     90960

[[47449     0]
 [43511     0]]
Ext Val Set F1: 0.0
Ext Val Set MCC: 0.0
Ext Val Set Accuracy: 0.48444444444444446
              precision    recall  f1-score   support

           0       0.48      1.00      0.65      2943
           1       0.00      0.00      0.00      3132

    accuracy                           0.48      6075
   macro avg       0.24      0.50      0.33      6075
weighted avg       0.23      0.48      0.32      6075

[[2943    0]
 [3132    0]]
FNA Set F1: 0.0
FNA Set MCC: 0.0
FNA Set Recall: 0.0
FNA Test Set Accuracy: 0.5021363907960451
              precision  

### 3.4.5. Voting

In [87]:
def Voting_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(XVal)
    print("Training Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, ensemble.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, ensemble.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [88]:
Voting_func()

Training Set F1: 0.5408562261624528
Training Set MCC: 0.396732768206014
Training Set Accuracy: 0.6780123131046614
              precision    recall  f1-score   support

           0       0.63      0.94      0.75     47449
           1       0.86      0.39      0.54     43511

    accuracy                           0.68     90960
   macro avg       0.74      0.67      0.65     90960
weighted avg       0.74      0.68      0.65     90960

[[44563  2886]
 [26402 17109]]
Ext Val Set F1: 0.7015330614851787
Ext Val Set MCC: 0.5949849771903635
Ext Val Set Accuracy: 0.7603292181069958
              precision    recall  f1-score   support

           0       0.67      0.99      0.80      2943
           1       0.98      0.55      0.70      3132

    accuracy                           0.76      6075
   macro avg       0.83      0.77      0.75      6075
weighted avg       0.83      0.76      0.75      6075

[[2905   38]
 [1418 1714]]
5F-CV: 0.7339920624288725
FNA Set F1: 0.550187545521609
FNA Se

# 4. Model Evaluation

## 4.1 Prediction

In [89]:
## deploy models for training set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d1\model3to1Voting_cbMSIn0d1.joblib")
#
## deploy model ##
predicted0n1_train = model_voting.predict(X_train)
#predicted0n1_trainProba = model_voting.predict_proba(X_train)         
trainDEFSDf["predicted0n1"] = predicted0n1_train
T_ = []
F_ = []
for i in range(len(trainDEFSDf)):
    if trainDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif trainDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
trainDEFSDf["T"] = T_
trainDEFSDf["F"] = F_
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d1\df_train_cbMSIn0d1_norm_0n1.csv"
trainDEFSDf.to_csv(savePath)

In [90]:
## deploy models for Ext Val set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d1\model3to1Voting_cbMSIn0d1.joblib")
#
## deploy model ##
predicted0n1_ext = model_voting.predict(X_val)
extDEFSDf["predicted0n1"] = predicted0n1_ext
T_ = []
F_ = []
for i in range(len(extDEFSDf)):
    if extDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif extDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
extDEFSDf["T"] = T_
extDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d1\df_ext_cbMSIn0d1_norm_0n1.csv"
extDEFSDf.to_csv(savePath)

In [91]:
## deploy models for FNA set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d1\model3to1Voting_cbMSIn0d1.joblib")
#
## deploy model ##
predicted0n1_FNA = model_voting.predict(X_FNA)
fnaDEFSDf["predicted0n1"] = predicted0n1_FNA
T_ = []
F_ = []
for i in range(len(fnaDEFSDf)):
    if fnaDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif fnaDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
fnaDEFSDf["T"] = T_
fnaDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d1\df_FNA_cbMSIn0d1_norm_0n1.csv"
fnaDEFSDf.to_csv(savePath)

In [92]:
## deploy models for DirectInfusion set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d1\model3to1Voting_cbMSIn0d1.joblib")
#
## deploy model ##
predicted0n1_Di = model_voting.predict(X_di)
diDEFSDf["predicted0n1"] = predicted0n1_Di
T_ = []
F_ = []
for i in range(len(diDEFSDf)):
    if diDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif diDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
diDEFSDf["T"] = T_
diDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d1\df_Di_cbMSIn0d1_norm_0n1.csv"
diDEFSDf.to_csv(savePath)